<a href="https://colab.research.google.com/github/jhaRahul2102/Emotion-detection-using-streamlit-and-keras/blob/main/Emotion_analysis_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d shawon10/ckplus

  0% 0.00/3.63M [00:00<?, ?B/s]
100% 3.63M/3.63M [00:00<00:00, 126MB/s]


In [ ]:
import zipfile,os

data=zipfile.ZipFile('/content/ckplus.zip')
data.extractall('/content/')

In [ ]:
import tensorflow as tf
import keras

from tensorflow.keras.optimizers import RMSprop

In [ ]:
model=keras.models.Sequential([
    keras.layers.Conv2D(16,(3,3),activation='relu',padding='same',input_shape=(48,48,1)),
    keras.layers.Conv2D(32,(3,3),activation='relu',padding='same'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(128,(3,3),activation='relu',padding='same'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(7,activation='softmax')
])

model.compile(loss='categorical_crossentropy',optimizer=RMSprop(learning_rate=0.001),metrics=['accuracy'])

In [ ]:


from keras.preprocessing.image import ImageDataGenerator

train_d=ImageDataGenerator(rescale=1.0/255.0,
                           rotation_range=60,
                           width_shift_range=0.2,
                           height_shift_range=0.2,
                           shear_range=0.2,
                           zoom_range=0.2,
                           horizontal_flip=True,
                           fill_mode='nearest')

valid_d=ImageDataGenerator(rescale=1.0/255.0)


#---------------------------------------------------------------------------

train_gen=train_d.flow_from_directory('/content/CK+48',target_size=(48,48),batch_size=25,class_mode='categorical',color_mode = "grayscale")




Found 981 images belonging to 7 classes.


In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau

early_stop=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
model_check=ModelCheckpoint(filepath='/content/mood2.h5',monitor='val_loss',save_best_only=True)
lr=ReduceLROnPlateau(monitor='val_loss',patience=3,factor=0.1)

In [ ]:
history=model.fit(train_gen,epochs=150,shuffle=True,callbacks=[early_stop,model_check,lr])

Epoch 1/150
40/40 [==============================] - 8s 196ms/step - loss: 0.2133 - accuracy: 0.9276 - lr: 0.0010
Epoch 2/150
40/40 [==============================] - 8s 196ms/step - loss: 0.2421 - accuracy: 0.9256 - lr: 0.0010
Epoch 3/150
40/40 [==============================] - 8s 196ms/step - loss: 0.2490 - accuracy: 0.9205 - lr: 0.0010
Epoch 4/150
40/40 [==============================] - 8s 197ms/step - loss: 0.2260 - accuracy: 0.9246 - lr: 0.0010
Epoch 5/150
40/40 [==============================] - 8s 197ms/step - loss: 0.2716 - accuracy: 0.9134 - lr: 0.0010
Epoch 6/150
40/40 [==============================] - 8s 197ms/step - loss: 0.2430 - accuracy: 0.9174 - lr: 0.0010
Epoch 7/150
40/40 [==============================] - 8s 194ms/step - loss: 0.2383 - accuracy: 0.9235 - lr: 0.0010
Epoch 8/150
40/40 [==============================] - 8s 194ms/step - loss: 0.2488 - accuracy: 0.9174 - lr: 0.0010
Epoch 9/150
40/40 [==============================] - 8s 196ms/step - loss: 0.2234 - accu

KeyboardInterrupt: ignored

In [ ]:
import joblib

joblib.dump(model,'emotion.pkl')

INFO:tensorflow:Assets written to: ram://f236d3a4-222f-4258-9f13-552cc6d6a05b/assets


['emotion.pkl']

In [ ]:
def processing(image,model):
  image=np.asarray(image)
  image_dims=np.expand_dims(image,axis=0)
  res=model.predict(image_dims,)
  return res 
model=tf.keras.models.load_model('/content/model.h5')
img=Image.open('/content/Training_266022.jpg')
img = cv2.cvtColor(np.float32(img), cv2.COLOR_RGB2BGR)
img = cv2.cvtColor(np.float32(img), cv2.COLOR_BGR2GRAY)
img= cv2.resize(img, (48, 48), interpolation=cv2.INTER_AREA)
result=processing(img,model)[0]
emotion_dict = ["Angry","Disgust","Fear","Happy","Neutral","Sad","Surprise"]
maxindex = int(np.argmax(result))
finalout = emotion_dict[maxindex]
output = str(finalout)
            


output